# Reason

In this tutorial we will use the pipeline `reason` to :
- identify spans that corresponds to the reason of hospitalisation
- check if there are named entities overlapping with my span of 'reason of hospitalisation'
- check for all named entities if they are tagged `is_reason`

In [ ]:
import spacy

import edsnlp.components

text = """COMPTE RENDU D'HOSPITALISATION du 11/07/2018 au 12/07/2018
MOTIF D'HOSPITALISATION
Monsieur Dupont Jean Michel, de sexe masculin, âgée de 39 ans, née le 23/11/1978, a été
hospitalisé du 11/08/2019 au 17/08/2019 pour attaque d'asthme. 

ANTÉCÉDENTS
Antécédents médicaux :
Premier épisode d'asthme en mai 2018."""

nlp = spacy.blank('fr')

# Extraction d'entités nommées
nlp.add_pipe(
    'eds.matcher',
    config=dict(
        terms=dict(respiratoire=[
            'asthmatique',
            'asthme',
            'toux',
        ])))


nlp.add_pipe("eds.normalizer")
nlp.add_pipe("eds.sections")
nlp.add_pipe("eds.reason", config=dict(use_sections=True))

doc = nlp(text)

The pipeline `reason` will add a key of spans called `reasons`. We check the first item in this list. 

In [2]:
reason = doc.spans['reasons'][0]
reason

hospitalisé du 11/08/2019 au 17/08/2019 pour attaque d'asthme.

Naturally, all spans included in `reasons` have the attribute `<span>._.is_reason` ==True


In [3]:
reason._.is_reason

True

In [4]:
# We chek if the span include named entities, their labels and the attribute is_reason
entities = reason._.ents_reason
for e in entities:
    print("Entity:",e.text,"-- Label:", e.label_, "-- is_reason:", e._.is_reason)

Entity: asthme -- Label: respiratoire -- is_reason: True


We can verify that named entities that do not overlap with the spans of reason, have their attribute `is_reason`==False

In [5]:
for e in doc.ents:
    print(e.start , e, e._.is_reason)

42 asthme True
54 asthme False
